In [ ]:
pip install pymongo

In [ ]:
import pymongo
import csv

from google.colab import userdata
my_db = userdata.get('mydatabase')

In [ ]:
client = pymongo.MongoClient(my_db)
db = client["bluesky"]
collection = db["posts"]

In [ ]:
csv_filename = "output_bluesky.csv"

# Campos extraídos para o csv
fields = [
    "uri",
    "record.$type",
    "record.createdAt",
    "record.text",
    "replyCount.$numberLong",
    "repostCount.$numberLong",
    "likeCount.$numberLong",
    "quoteCount.$numberLong"
]

In [ ]:
# Garante que não irá adicionar textos duplicados
existing_texts = set()

In [ ]:
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)


    writer.writerow(fields)


    for document in collection.find():

        record_text = document.get("record", {}).get("text", "")


        if record_text and record_text not in existing_texts:

            existing_texts.add(record_text)

            def safe_get(field, subfield):
                value = document.get(field, {})
                return value.get(subfield, "") if isinstance(value, dict) else value

            row = [
                document.get("uri", ""),
                document.get("record", {}).get("$type", ""),
                document.get("record", {}).get("createdAt", ""),
                record_text,
                safe_get("replyCount", "$numberLong"),
                safe_get("repostCount", "$numberLong"),
                safe_get("likeCount", "$numberLong"),
                safe_get("quoteCount", "$numberLong")
            ]

            writer.writerow(row)